# Importing Libraries and Dataset

In [ ]:
#Import Libraries
import pandas as pd
import numpy as np

from sklearn import metrics as skmetrics

import keras
import tensorflow
import tensorflow as tf
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Input, Embedding, SimpleRNN, Dropout, LSTM, Bidirectional, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam, SGD
from keras import layers

In [ ]:
#Import Data

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [ ]:
X_smote = pd.read_csv('drive/MyDrive/BT4012/india fraud dataset/data/X_smote.csv')
y_smote = pd.read_csv('drive/MyDrive/BT4012/india fraud dataset/data/y_smote.csv')
X_val = pd.read_csv('drive/MyDrive/BT4012/india fraud dataset/data/X_val.csv')
y_val = pd.read_csv('drive/MyDrive/BT4012/india fraud dataset/data/y_val.csv')
X_test = pd.read_csv('drive/MyDrive/BT4012/india fraud dataset/data/X_test.csv')
y_test = pd.read_csv('drive/MyDrive/BT4012/india fraud dataset/data/y_test.csv')

In [ ]:
# X_smote = pd.read_csv('data/X_smote.csv')
# y_smote = pd.read_csv('data/y_smote.csv')
# X_val = pd.read_csv('data/X_val.csv')
# y_val = pd.read_csv('data/y_val.csv')
# X_test = pd.read_csv('data/X_test.csv')
# y_test = pd.read_csv('data/y_test.csv')

# Evaluation Metrics Function

In [ ]:
def print_statistics(y_actual, y_pred, y_prob, dataset_type):
  print(f"====================={dataset_type}====================")
  print(f"accuracy: {round(skmetrics.accuracy_score(y_actual, y_pred),5)}")
  print(f"precision (macro): {round(skmetrics.precision_score(y_actual, y_pred, average='macro'),5)}")
  print(f"recall (macro): {round(skmetrics.recall_score(y_actual, y_pred, average='macro'),5)}")
  print(f"f1 score (macro): {round(skmetrics.f1_score(y_actual, y_pred, average='macro'),5)}")
  print(f"f1 score of class 1: {round(skmetrics.f1_score(y_actual, y_pred, pos_label=1),5)}")
  print(f"f1 score of class 0: {round(skmetrics.f1_score(y_actual, y_pred, pos_label=0),5)}")
  prec, recall, thresholds = skmetrics.precision_recall_curve(y_actual, y_prob)
  print(f"pr auc score of class 1: {round(skmetrics.auc(recall, prec),5)}")
  prec_0, recall_0, thresholds = skmetrics.precision_recall_curve(y_actual, y_prob, pos_label=0)
  print(f"pr auc score of class 0: {round(skmetrics.auc(recall_0, prec_0),5)}")

# Neural Network Models

## Basic NN

In [ ]:
# MODEL FOR 5 LAYERS
def train_base_model(n, dropout, X_train, y_train, X_val, y_val):
    model = Sequential()
    model.add(Dense(units=n, activation='relu', input_dim=130))
    
    model.add(Dropout(dropout)) 
    model.add(Dense(n, activation='relu')) # 1 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(n/2, activation='relu')) # 2 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(n/4, activation='relu')) # 3 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(n/6, activation='relu')) # 4 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(n/8, activation='relu')) # 5 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(n/16, activation='relu')) # 6 hidden layer 
    model.add(Dropout(dropout)) 
    model.add(Dense(units=1, activation='sigmoid')) #output layer sigmoid as binary
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy", metrics.Precision(),metrics.Recall()])
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    model.fit(X_train, y_train, batch_size=128, epochs=10, callbacks=[callback], validation_data=(X_val, y_val))
    return model

In [ ]:
model_train = train_base_model(n=4080, dropout=0.1, X_train=X_smote, y_train=y_smote, X_val= X_val, y_val=y_val)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4080)              534480    
                                                                 
 dropout (Dropout)           (None, 4080)              0         
                                                                 
 dense_1 (Dense)             (None, 4080)              16650480  
                                                                 
 dropout_1 (Dropout)         (None, 4080)              0         
                                                                 
 dense_2 (Dense)             (None, 2040)              8325240   
                                                                 
 dropout_2 (Dropout)         (None, 2040)              0         
                                                                 
 dense_3 (Dense)             (None, 1020)              2

In [ ]:
# Evaluating Train and Test
train_pred_proba = model_train.predict(X_smote)
train_pred = np.round(train_pred_proba)
val_pred_proba = model_train.predict(X_val)
val_pred = np.round(val_pred_proba)
test_pred_proba = model_train.predict(X_test)
test_pred = np.round(test_pred_proba)

91/91 [==============================] - 0s 2ms/step


In [ ]:
print_statistics(y_smote, train_pred, train_pred_proba, 'train')
print_statistics(y_val, val_pred, val_pred_proba, 'val')
print_statistics(y_test, test_pred, test_pred_proba, 'test')

=====================train====================
accuracy: 0.55421
precision (macro): 0.6449
recall (macro): 0.55421
f1 score (macro): 0.47153
f1 score of class 1: 0.68057
f1 score of class 0: 0.26249
pr auc score of class 1: 0.28189
pr auc score of class 0: 0.69056
=====================val====================
accuracy: 0.37136
precision (macro): 0.59563
recall (macro): 0.55399
f1 score (macro): 0.35833
f1 score of class 1: 0.44977
f1 score of class 0: 0.26688
pr auc score of class 1: 0.15601
pr auc score of class 0: 0.82875
=====================test====================
accuracy: 0.37795
precision (macro): 0.60269
recall (macro): 0.55972
f1 score (macro): 0.36587
f1 score of class 1: 0.45338
f1 score of class 0: 0.27836
pr auc score of class 1: 0.15679
pr auc score of class 0: 0.82597
